<a href="https://colab.research.google.com/github/yuri-pessoa/gurobi-exemplos/blob/main/problema_transportes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Iniciando com Gurobi
Primeiramente devemos instalar o Gurobi Python API com o comando `` %pip install gurobipy ``

Esta aplicação fornece uma licença provisória do solver Gurobi, com recursos limitados. Para utilizar a licença com recursos completos, você deve aplicar para uma licença acadêmica no site.

In [ ]:
%pip install gurobipy

Agora importamos as bibliotecas que iremos utilizar. Note que o ``gurobipy`` é importado como uma biblioteca do Python.

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

# Descrição do problema

Neste exemplo temos um problema de produção e distribuição, onde queremos determinar a produção ótima de dois produtos, canetas e lápis, a partir de duas fábricas.<br>Em seguida, devemos determinar a distribuição das canetas e lápis entre as fábricas e os centros de distribuição, de modo a atender a demanda. <br>Cada fábrica possui custos de produção que variam conforme a cidade a ser atendida. Cada cidade possui uma demanda que deve ser atendida.

### Conjuntos

- $P = \{\texttt{Canetas, Lápis}\} \quad\quad\quad\quad \texttt{Produtos}$

- $F = \{\texttt{Fábrica 1,Fábrica 2}\} \quad\quad\quad\quad \texttt{Fábricas}$

- $C = \{\texttt{Cidade 1, Cidade 2, Cidade 3}\} \quad\quad\quad\quad \texttt{Cidades}$

### Parâmetros

- $p_f$ é a capacidade de produção em cada fábrica $f$, $ \space \forall f \in F$
- $d_c$ é a demanda em cada cidade $c$, $ \space \forall c \in C$
- $c_{p,f,c}$ é o custo de enviar um produto $p$ da fábrica $f$ até a cidade $c$, $\forall p \in P, \space \forall f \in F, \space \forall c \in C$
- $x_{p, f}$ é a quantidade de produtos $p$ produzidos na fábrica $f$, $\forall p \in P, \space \forall f \in F$
- $x_{p, c}$ é a quantidade de produtos $p$ recebidos na cidade $c$, $\forall p \in P, \space \forall c \in C$

### Restrições
$$
\begin{align*}
\sum_{p}x_{p, f, c} &\le p_f, &\forall f \in F \quad \quad \quad &\texttt{produção[p]}\\
\sum_{d}x_{p, f, c} &\ge d_c, &\forall c \in C \quad \quad \quad &\texttt{demanda[d]}\\
x_{p, f} \space = \space x_{p, c}, &\space &\forall p \in P, \forall f \in F, \forall c \in C &\space\texttt{conservação[p, d]}\\
\end{align*}
$$

Iniciamos a modelagem com a criação dos conjuntos e parâmetros deste problema.


In [ ]:
produtos = ['Canetas', 'Lápis']

nós = ['Fábrica 1', 'Fábrica 2', 'Cidade 1', 'Cidade 2', 'Cidade 3']

arcos, capacidade = gp.multidict({
    ('Fábrica 1', 'Cidade 1'): 100,
    ('Fábrica 1', 'Cidade 2'): 80,
    ('Fábrica 1', 'Cidade 3'): 120,
    ('Fábrica 2', 'Cidade 1'): 120,
    ('Fábrica 2', 'Cidade 2'): 120,
    ('Fábrica 2', 'Cidade 3'): 120
})

custos = {
    ('Canetas', 'Fábrica 1', 'Cidade 1'): 10,
    ('Canetas', 'Fábrica 1', 'Cidade 2'): 20,
    ('Canetas', 'Fábrica 1', 'Cidade 3'): 60,
    ('Canetas', 'Fábrica 2', 'Cidade 1'): 40,
    ('Canetas', 'Fábrica 2', 'Cidade 2'): 40,
    ('Canetas', 'Fábrica 2', 'Cidade 3'): 30,
    ('Lápis', 'Fábrica 1', 'Cidade 1'): 20,
    ('Lápis', 'Fábrica 1', 'Cidade 2'): 20,
    ('Lápis', 'Fábrica 1', 'Cidade 3'): 80,
    ('Lápis', 'Fábrica 2', 'Cidade 1'): 60,
    ('Lápis', 'Fábrica 2', 'Cidade 2'): 70,
    ('Lápis', 'Fábrica 2', 'Cidade 3'): 30
}

# valores positivos indica produção nas fábricas, valores negativos indica demanda nas cidades
produção = {
    ('Canetas', 'Fábrica 1'): 60,
    ('Canetas', 'Fábrica 2'): 50,
    ('Lápis', 'Fábrica 1'): 60,
    ('Lápis', 'Fábrica 2'): 40,
    ('Canetas', 'Cidade 1'): -50,
    ('Canetas', 'Cidade 2'): -50,
    ('Canetas', 'Cidade 3'): -10,
    ('Lápis', 'Cidade 1'): -40,
    ('Lápis', 'Cidade 2'): -30,
    ('Lápis', 'Cidade 3'): -30
}


Nesta seção criamos o modelo MIP invocando o ``gurobipy`` e ao final geramos a sua solução utilizando a instrução ``modelo.optimize()``.



In [ ]:
# iniciamos invocando a função try, que condiciona a execução do código a ocorrência de erros durante a criação do modelo. Caso não encontre erros o código é executado
try:
# invoca o comando gp.Model para criar e nomear o modelo
    modelo = gp.Model('rede logística multiproduto')

# invoca o comando addVars para criar a função objetivo e as variáveis decisórias
    fluxo = modelo.addVars(produtos, arcos, obj=custos, name='fluxo')

# visualiza as variáveis decisórias e sua quantidade
    #print(fluxo)
    #print(type(fluxo))
    #print(len(fluxo))

# crias as restrições de capacidade de produção
    r1 = modelo.addConstrs(
        (fluxo.sum('*', i, j) <= capacidade[i, j] for i, j in arcos), 'capacidade')

# cria as restrições de conservação de fluxo
    r2 = modelo.addConstrs((fluxo.sum( p, '*', j) + produção[p, j] == fluxo.sum(p, j, '*')
        for p in produtos for j in nós), 'conservação')

# invoca o comando para otimizar o modelo minimizando os custos por padrão
    modelo.optimize()
# gerando as exceções para identificarmos erros no código
except gp.GurobiError as e:
    print('Temos um código de erro' + str(e.errno) + ':' + str(e))

except Exception as e:
    print('Ops, temos um erro de atributo: {:s} '.format(str(e)))


Imprimindo a solução do problema de transporte



In [ ]:
# imprimindo a solução
def imprime_solução():
    if modelo.Status==GRB.OPTIMAL:
        print('Solução ótima encontrada:')
        pd.solution = modelo.getAttr('X', fluxo)
        for p in produtos:
            print('Fluxo ótimo de {:s}: '.format(p), sep='\n')
            for i, j in arcos:
                if pd.solution[p, i, j] > 0:
                    print('{:s} -> {:s}: {:g}'.format(i, j, pd.solution[p, i, j]))
    else:
        print('Solução subótima encontrada: ')
        pd.solution = modelo.getAttr('X', fluxo)
        for p in produtos:
            print('Fluxo ótimo de {:s}:'.format(p), sep='\n')
            for i, j in arcos:
                if pd.solution[p, i, j] > 0:
                    print('{:s} -> {:s}: {:g}'.format(i, j, pd.solution[p, i, j]))


imprime_solução()

# Adicionando uma terceira restrição ao modelo

Recentemente o diretor de produção da empresa decidiu que cada fábrica deverá enviar para cada cidade a quantidade mínima de 12 caixas de produtos, compartilhadas entre canetas e lápis. Isso quer dizer que agora temos uma quantidade mínima de caixas a serem enviadas para cada uma das três cidades.

$$
\begin{align*}
\sum_{p}x_{f,c} \space \ge \space 12, &\space \forall f \in F \space \forall c \in C \quad \quad \quad &\texttt{envio}\_\texttt{minimo[f, c]}\end{align*}
$$

In [ ]:
r3 = modelo.addConstrs((fluxo.sum('*', i, j) >= 12 for i, j in arcos), name='envio_minimo')
modelo.update()

In [ ]:
# gerando a solução após a inclusão da terceira restrição
modelo.optimize()


In [ ]:
# imprimindo a segunda solução do modelo
imprime_solução()

Chegamos ao fim deste exemplo! <br>
Compare as diferenças entre a primeira e a segunda solução do modelo. O que mudou?
